# Install Required Libraries

In [ ]:
!pip install langchain langchain_community langchain_chroma

In [ ]:
! pip install langchain-cohere

In [ ]:
! pip install -qU pypdf langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

llm = ChatCohere(model="")

··········


# Load PDFs from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/climate_change"

FireSmart-Guide_2024.pdf  SynthesisReport_EN.pdf


## Indexing: Loading documents from PDF

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/drive/My Drive/climate_change/FireSmart-Guide_2024.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

36


In [ ]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)



FireSmart, Intelli-feu and other associated Marks are trademarks of the Canadian Interagency Forest 
{'source': '/content/drive/My Drive/climate_change/FireSmart-Guide_2024.pdf', 'page': 0}


## Indexing: Splitting Documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

47

In [ ]:
all_splits[10].metadata

{'source': '/content/drive/My Drive/climate_change/FireSmart-Guide_2024.pdf',
 'page': 8,
 'start_index': 0}

## Indexing: Storing Step

In [ ]:
from langchain_chroma import Chroma
from langchain_cohere import CohereEmbeddings

cohere_embeddings = CohereEmbeddings(model="embed-multilingual-v3.0")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=CohereEmbeddings())

In [ ]:
vectorstore

# Retrieval

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are some ways to save yourself from fire?")

len(retrieved_docs)

6

In [ ]:
print(retrieved_docs[0].page_content)

•  
 Space tr
ees 3 metres from branch tips to reduce the intensity and rate of 
spread of fire.
•  
 Remove smaller evergr
een trees that can act as a ladder for fire to 
           
move into the tr
eetops.
•  
 Clean woody debris and combustible shrubs fr
om the ground.Large yards
30 m+
In an emergency, you and your family may need to leave your neighbourhood 
while emergency responders enter. In order for this to happen safely and efficiently, consider the following tips:
•  
 Clearly mark your addr
ess.
•  
 Clear vegetation fr
om access routes to and from your home. Target trees and 
branches that would make it difficult for a fire truck to approach your home.
•  
 If you have a large pr
operty, make sure your driveway has a spot to turn 
around and, when possible, provide two access routes to your home.Roadways and driveways
25


# Retrieval and Generation

In [ ]:
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering related to climate change. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What are some ways to save yourself from fire?"})
print(response["answer"])

To protect yourself from fire, prepare an emergency plan and act quickly. Here are some ways to increase your chances of safety:

1. Clearly mark your address to help responders find you.
2. Ensure access routes are clear of vegetation, especially near your home.
3. Have a large property? Make sure your driveway has a turnaround spot and, if possible, two access routes.


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering related to climate change in Hindi."
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "आग लगने के समय अपने चेहरे को kaise बचाएं?"})
print(response["answer"])

अगर आग लगने की स्थिति में, चेहरे को बचाने के लिए सबसे अच्छा तरीका एक तौलिया या कोई भी सूखा कपड़ा लेकर अपने मुंह और नाक को कोसना है। यह ऑक्सीजन लेने में मदद करेगा और चेहरे पर आग लगने से बचेगा। इसके अलावा, आग से दूर भागने की कोशिश करें और यदि संभव हो तो अन्य लोगों को सुरक्षित स्थान पर ले जाएँ।
